In [1]:
import requests 
from bs4 import BeautifulSoup as bs 
import pandas as pd

In [2]:
r=requests.get('https://cryptoslam.io/')
content = bs(r.content)

# print(content.prettify())

In [3]:
table=content.select('table.table-hover.js-top-by-sales-table-all.summary-sales-table')[0]
columns = table.find("thead").find_all("th")
column_names = [c.string for c in columns]

table_rows = table.find('tbody').find_all('tr')

l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [str(tr.get_text()).strip() for tr in td]
    l.append(row)

q = []
for tr in table_rows:
    td = tr.select('td span.summary-sales-table__column-product-name')
    row = [str(tr.get_text()).strip() for tr in td]
    q.append(row)


df1 = pd.DataFrame(q, columns=['Collections'])
df1 = df1.loc[~df1.index.duplicated(keep='first')]
df2 = pd.DataFrame(l, columns=column_names)
df2 = df2.loc[~df2.index.duplicated(keep='first')]

nft_all_time = df1.join(df2)
nft_all_time.drop(nft_all_time.columns[[2,3]],axis=1,inplace=True)
nft_all_time.head(100)

,Collections,Sales,Buyers,Txns,Owners
0,Axie Infinity,"$3,391,483,898","1,105,095","9,995,287","2,682,388"
1,CryptoPunks,"$1,676,251,328","4,731","18,987","3,293"
2,Art Blocks,"$1,078,767,070","21,149","118,416","25,468"
3,Bored Ape Yacht Club,"$789,907,069","8,306","22,612","5,861"
4,NBA Top Shot,"$785,818,188","375,813","11,965,958","605,878"
...,...,...,...,...,...
95,Galaxy Fight Club,"$18,018,304","4,242","13,935","2,465"
96,Solsteads,"$17,979,292","1,630","3,260","1,643"
97,Forgotten Runes,"$17,928,970","3,635","11,595","2,482"
98,CryptoArte,"$17,819,905","4,123","9,712","3,351"


In [4]:
nft_all_time.to_csv('nft_sales.csv',index=False)